In [1]:
!pip install langchain
!pip install langchain-groq

import os
import json
import requests
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain.agents import create_tool_calling_agent, AgentExecutor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.4 MB/s eta 0:00:00


In [5]:
from google.colab import userdata

SECTORS_API_KEY = userdata.get('SECTORS_API_KEY')
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

### Warm up
Here's a quick warm-up exercise. Below, we're using one of the [endpoints available at Sectors API](https://docs.sectors.app) to make our first HTTP request.  

In [7]:
import requests
import pprint

headers = {
    "Authorization": SECTORS_API_KEY
}


def get_overview(stock:str, section:str) -> requests:
    url = f"https://api.sectors.app/v1/company/report/{stock}/?sections={section}"
    response = requests.get(url, headers=headers)
    return response


response = get_overview("BBCA", "financials")
pprint.pprint(response.json())

{'company_name': 'PT Bank Central Asia Tbk.',
 'financials': {'cash_flow_debt_ratio': 26.30499465863844,
                'dar_mrq': 0.00144476996394943,
                'der_mrq': 0.00855015750574211,
                'historical_financials': [{'cash_and_equivalents': None,
                                           'cash_only': None,
                                           'current_liabilities': None,
                                           'earnings': 25855154000000,
                                           'earnings_before_tax': 32706064000000,
                                           'fixed_assets': None,
                                           'gross_profit': None,
                                           'operating_pnl': 32512504000000,
                                           'revenue': 63028090000000,
                                           'tax': 6854404000000,
                                           'total_assets': 824787944000000,
                      

In [4]:
import requests
import pprint

headers = {
    "Authorization": SECTORS_API_KEY
}


def get_most_traded_stock(start_date: str, end_date: str, top_n: int = 5) -> requests:
    url = f"https://api.sectors.app/v1/most-traded/?start={start_date}&end={end_date}&n_stock={top_n}"
    print(url)
    response = requests.get(url, headers=headers)
    return response


response = get_most_traded_stock("2024-08-05", "2024-08-05")
pprint.pprint(response.json())

https://api.sectors.app/v1/most-traded/?start=2024-08-05&end=2024-08-05&n_stock=5
{'2024-08-05': [{'company_name': 'PT GoTo Gojek Tokopedia Tbk',
                 'price': 51,
                 'symbol': 'GOTO.JK',
                 'volume': 5466579700},
                {'company_name': 'PT Bukalapak.com Tbk',
                 'price': 109,
                 'symbol': 'BUKA.JK',
                 'volume': 768889200},
                {'company_name': 'PT Astrindo Nusantara Infrastruktur Tbk.',
                 'price': 59,
                 'symbol': 'BIPI.JK',
                 'volume': 553985600},
                {'company_name': 'PT Era Media Sejahtera Tbk',
                 'price': 51,
                 'symbol': 'DOOH.JK',
                 'volume': 425860800},
                {'company_name': 'Bumi Resources Tbk',
                 'price': 72,
                 'symbol': 'BUMI.JK',
                 'volume': 380020800}]}


# Tools and Function

In [8]:
def get_today_date() -> str:
    """
    Get today's date
    """
    from datetime import date

    today = date.today()
    return today.strftime("%Y-%m-%d")

In [9]:
from langchain_core.tools import tool

def retrieve_from_endpoint(url: str) -> dict:
    headers = {"Authorization": SECTORS_API_KEY}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.HTTPError as err:
        raise SystemExit(err)
    return json.dumps(data)


@tool
def get_top_companies_by_tx_volume(
    start_date: str, end_date: str, top_n: int = 5
) -> str:
    """
    Get top companies by transaction volume
    """
    url = f"https://api.sectors.app/v1/most-traded/?start={start_date}&end={end_date}&n_stock={top_n}"

    return retrieve_from_endpoint(url)

@tool
def get_company_overview(stock: str) -> str:
    """
    Get the overview of a stock, including email, address, phone number, market cap information, listing date etc
    """
    url = f"https://api.sectors.app/v1/company/report/{stock}/?sections=overview"
    print(url)

    return retrieve_from_endpoint(url)


@tool
def get_daily_tx(stock: str, start_date: str, end_date: str) -> str:
    """
    Get daily transaction for a stock
    """
    url = f"https://api.sectors.app/v1/daily/{stock}/?start={start_date}&end={end_date}"
    print(url)

    return retrieve_from_endpoint(url)


@tool
def get_performance_since_ipo(stock: str) -> str:
  """
  Get stock performance after IPO listing
  """
  url = f"https://api.sectors.app/v1/listing-performance/{stock}/"
  print(url)

  return retrieve_from_endpoint(url)

### Self Assessment

Complete **any one of the following two** exercises to get a certificate.



# Exercise 1
Modify and improve the orchestration script below. Here are some things you might want to try:



*   Sam says [the `llama3-groq-70b-8192-tool-use-preview`](https://console.groq.com/docs/models) is a superior model specialized in tool use and function calling tasks. It might be worth swapping in that model.
*   Could the system prompt be improved?
*   Could the docstring in that tool be improved?



In [10]:
tools = [get_top_companies_by_tx_volume]
llm = ChatGroq(
    temperature=0,
    model_name="llama3-groq-70b-8192-tool-use-preview",
    groq_api_key=GROQ_API_KEY,
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are the financial robo-advisor, answer the following queries like a Financial Agent,
            being as factual and analytical as possible. Do not make up stuff.
            Try to cite your number as much as possible.
            Your get_daily_tx tool will need the start and end dates, and if you need the start and end dates
            and they are not explicitly provided, please infer from the query.
            If the volume was about a single day (such as specific single date or yesterday),
            the start date and end date parameter should be the same date. Start and end date will be in the format YYYY-MM-DD.
            Whenever you return a list of names, return also the corresponding values for each name.
            Today's date is
            """
            + get_today_date(),
        ),
        ("human", "{input}"),
        # msg containing previous agent tool invocations and corresponding tool outputs
        MessagesPlaceholder("agent_scratchpad"),
    ]
)
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

query = "What are the top 3 companies by transaction volume over the last 7 days?"
result = agent_executor.invoke({"input": query})
print("Answer:", "\n", result["output"], "\n\n======\n\n")





> Entering new AgentExecutor chain...

Invoking: `get_top_companies_by_tx_volume` with `{'start_date': '2024-10-01', 'end_date': '2024-10-07', 'top_n': 3}`


{"2024-10-01": [{"symbol": "BUMI.JK", "company_name": "Bumi Resources Tbk", "volume": 4971377700, "price": 143}, {"symbol": "GOTO.JK", "company_name": "PT GoTo Gojek Tokopedia Tbk", "volume": 2147891100, "price": 67}, {"symbol": "BRMS.JK", "company_name": "Bumi Resources Minerals Tbk", "volume": 1890329500, "price": 250}], "2024-10-02": [{"symbol": "BUMI.JK", "company_name": "Bumi Resources Tbk", "volume": 7989500200, "price": 138}, {"symbol": "GOTO.JK", "company_name": "PT GoTo Gojek Tokopedia Tbk", "volume": 5047793900, "price": 65}, {"symbol": "BRMS.JK", "company_name": "Bumi Resources Minerals Tbk", "volume": 2530774500, "price": 268}], "2024-10-03": [{"symbol": "GOTO.JK", "company_name": "PT GoTo Gojek Tokopedia Tbk", "volume": 3526729900, "price": 64}, {"symbol": "BUMI.JK", "company_name": "Bumi Resources Tbk", "volume": 1

If you'd like another level of challenge, try and apply your tool-calling LLM model on the following, arguably more challenging, queries.

You need to successfully execute this exercise with these 3 queries returning correct results to be considered successful at this challenge!

In [ ]:
queries = [
    "What are the top 5 companies by transaction volume on the first of this month?",
    "What are the most traded stock yesterday?",
    "What are the top 7 most traded stocks between 6th June to 10th June this year?",
]

for query in queries:
    print("Question:", query)
    result = agent_executor.invoke({"input": query})
    print("Answer:", "\n", result["output"], "\n\n======\n\n")

Question: What are the top 5 companies by transaction volume on the first of this month?


> Entering new AgentExecutor chain...

Invoking: `get_top_companies_by_tx_volume` with `{'start_date': '2024-08-01', 'end_date': '2024-08-01', 'top_n': 5}`


{"2024-08-01": [{"symbol": "GOTO.JK", "company_name": "PT GoTo Gojek Tokopedia Tbk", "volume": 1425057600, "price": 53}, {"symbol": "BIPI.JK", "company_name": "PT Astrindo Nusantara Infrastruktur Tbk.", "volume": 521955600, "price": 61}, {"symbol": "NICL.JK", "company_name": "PT PAM Mineral Tbk", "volume": 362601100, "price": 252}, {"symbol": "WIKA.JK", "company_name": "PT Wijaya Karya (Persero) Tbk", "volume": 314787400, "price": 220}, {"symbol": "DEWA.JK", "company_name": "Darma Henwa Tbk", "volume": 294681400, "price": 67}]}The top 5 companies by transaction volume on the first of this month are:
1. PT GoTo Gojek Tokopedia Tbk with a volume of 1,425,057,600.
2. PT Astrindo Nusantara Infrastruktur Tbk. with a volume of 521,955,600.
3. PT P

# Exercise 2

Our users have been using our tool and having a lot of success with it. It is now time to take it one step further: by collecting user's feedback!

Here are the sample queries that performed sub-optimally according to user feedback:


*   What is the performance of GOTO (symbol: GOTO) since its IPO listing?
*   If i had invested into GOTO vs BREN on their respective IPO listing date, which one would have given me a better return over a 90 day horizon?"

Clearly, it's time to expand the tools that our language model has access to. For the second exercise, you'd be implementing **an additional** tool directly into the orchestrator to give your financial AI model a direct pathway to answering questions relating to stock performance since their listing date.


> You may have to [refer to Sectors API Documentation](https://docs.sectors.app) for a list of endpoints and pick the one most suitable for the job.

To help you get started, I've also added two new tools. Use them as a base reference! If you proceed to run the exercise without adding the right tool(s), `query_4` and `query_5` is most likely going to fail or cause the LLM to answer incorrectly.

In [ ]:
tools = [
    get_company_overview,
    # we created this in the earlier code chunk under Exercise 1
    # (so make sure you've run that cell),
    get_top_companies_by_tx_volume,
    get_daily_tx,
    get_performance_since_ipo
]

query_1 = "What are the top 3 companies by transaction volume over the last 7 days?"
query_2 = "Based on the closing prices of BBCA between 1st and 30th of June 2024, are we seeing an uptrend or downtrend? Try to explain why."
query_3 = "What is the company with the largest market cap between BBCA and BREN? For said company, retrieve the email, phone number, listing date and website for further research."
query_4 = "What is the performance of GOTO (symbol: GOTO) since its IPO listing?"
query_5 = "If i had invested into GOTO vs BREN on their respective IPO listing date, which one would have given me a better return over a 90 day horizon?"

queries = [query_1, query_2, query_3, query_4, query_5]

llm = ChatGroq(
    temperature=0,
    model_name="llama3-groq-70b-8192-tool-use-preview",
    groq_api_key=GROQ_API_KEY,
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"""
            You are the financial robo-advisor, answer the following queries like a Financial Agent,
            being as factual and analytical as possible. Do not make up stuff.
            Try to cite your number as much as possible. Retrieve and show the corresponding data first
            such as the market cap number or data that correspond to the question before you make conclusion and show the answer.
            Your get_daily_tx tool will need the start and end dates, and if you need the start and end dates
            and they are not explicitly provided, please infer from the query.
            If the volume was about a single day (such as specific single date or yesterday),
            the start date and end date parameter should be the same date. Start and end date will be in the format YYYY-MM-DD.
            Whenever you return a list of names, return also the corresponding values for each name.

            For most of your queries such as company performance since the date of its initial public offering (IPO),
            these information should be available in your get_performance_since_ipo tool,
            but the listing performance data is accessible only for stocks listed after May 2005.
            Note that the endpoint for performance since IPO has only one required parameter, which is the stock.
            Every time you return the symbol of the stock, result also the number in question in a nice tabular format.

            Today's date is {get_today_date()}.
            """
        ),
        ("human", "{input}"),
        # msg containing previous agent tool invocations and corresponding tool outputs
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

for i, query in enumerate(queries):
    print(f"Question {i+1}:", query)
    result = agent_executor.invoke({"input": query})
    print("Answer:", "\n", result["output"], "\n\n======\n\n")

Question 1: What are the top 3 companies by transaction volume over the last 7 days?

Invoking: `get_top_companies_by_tx_volume` with `{'start_date': '2024-09-17', 'end_date': '2024-09-24', 'top_n': 3}`


{"2024-09-17": [{"symbol": "GOTO.JK", "company_name": "PT GoTo Gojek Tokopedia Tbk", "volume": 7405920400, "price": 65}, {"symbol": "BUMI.JK", "company_name": "Bumi Resources Tbk", "volume": 1635899400, "price": 105}, {"symbol": "BSBK.JK", "company_name": "PT Wulandari Bangun Laksana Tbk", "volume": 838429600, "price": 57}], "2024-09-18": [{"symbol": "GOTO.JK", "company_name": "PT GoTo Gojek Tokopedia Tbk", "volume": 8963789100, "price": 61}, {"symbol": "BSBK.JK", "company_name": "PT Wulandari Bangun Laksana Tbk", "volume": 1326772400, "price": 58}, {"symbol": "BUMI.JK", "company_name": "Bumi Resources Tbk", "volume": 1241907400, "price": 105}], "2024-09-19": [{"symbol": "GOTO.JK", "company_name": "PT GoTo Gojek Tokopedia Tbk", "volume": 8638630900, "price": 63}, {"symbol": "BUMI.JK",

The top 3 companies by transaction volume over the last 7 days are:

1. PT GoTo Gojek Tokopedia Tbk (GOTO.JK) with a total volume of 37,439,990,000
2. Bumi Resources Tbk (BUMI.JK) with a total volume of 8,439,990,000
3. PT Wulandari Bangun Laksana Tbk (BSBK.JK) with a total volume of 2,506,720,000

> Finished chain.
Answer: 
 The top 3 companies by transaction volume over the last 7 days are:

1. PT GoTo Gojek Tokopedia Tbk (GOTO.JK) with a total volume of 37,439,990,000
2. Bumi Resources Tbk (BUMI.JK) with a total volume of 8,439,990,000
3. PT Wulandari Bangun Laksana Tbk (BSBK.JK) with a total volume of 2,506,720,000 



Question 2: Based on the closing prices of BBCA between 1st and 30th of June 2024, are we seeing an uptrend or downtrend? Try to explain why.

Invoking: `get_daily_tx` with `{'stock': 'BBCA', 'start_date': '2024-06-01', 'end_date': '2024-06-30'}`
responded: To determine if there is an uptrend or downtrend in BBCA's closing prices between June 1st and June 30th, 2024,

Based on the closing prices of BBCA between June 1st and June 30th, 2024, we can observe a general uptrend. The closing price started at $9275 on June 3rd and ended at $9925 on June 28th, showing an increase of $650. This indicates a positive trend in the stock's performance during the specified period.

> Finished chain.
Answer: 
 Based on the closing prices of BBCA between June 1st and June 30th, 2024, we can observe a general uptrend. The closing price started at $9275 on June 3rd and ended at $9925 on June 28th, showing an increase of $650. This indicates a positive trend in the stock's performance during the specified period. 



Question 3: What is the company with the largest market cap between BBCA and BREN? For said company, retrieve the email, phone number, listing date and website for further research.

Invoking: `get_company_overview` with `{'stock': 'BBCA'}`


https://api.sectors.app/v1/company/report/BBCA/?sections=overview
{"symbol": "BBCA.JK", "company_name": "PT Bank C

Based on the information retrieved, PT Bank Central Asia Tbk. (BBCA) has a larger market cap than PT Barito Renewables Energy Tbk. (BREN). Here are the details for PT Bank Central Asia Tbk.:

- Address: Menara BCA, Grand Indonesia, Jalan MH Thamrin No. 1, Jakarta 10310
- Email: investor_relations@bca.co.id
- Phone: 021-23588000
- Website: www.bca.co.id
- Market Cap: 1,340,615,656,720
- Market Cap Rank: 1
- Industry: Banks
- Sector: Financials
- Sub-Industry: Banks
- Sub-Sector: Banks
- Employee Number: 27,547
- Listing Board: Main
- Listing Date: 2000-05-31
- Last Close Price: 10,950
- Latest Close Date: 2024-09-23
- Daily Close Change: 0.0162412993039443

> Finished chain.
Answer: 
 Based on the information retrieved, PT Bank Central Asia Tbk. (BBCA) has a larger market cap than PT Barito Renewables Energy Tbk. (BREN). Here are the details for PT Bank Central Asia Tbk.:

- Address: Menara BCA, Grand Indonesia, Jalan MH Thamrin No. 1, Jakarta 10310
- Email: investor_relations@bca.co.id

The performance of GOTO (symbol: GOTO) since its IPO listing is as follows:
- Change in the last 30 days: -0.343195
- Change in the last 365 days: -0.707101
- Change in the last 7 days: 0.118343
- Change in the last 90 days: 0.035503

> Finished chain.
Answer: 
 The performance of GOTO (symbol: GOTO) since its IPO listing is as follows:
- Change in the last 30 days: -0.343195
- Change in the last 365 days: -0.707101
- Change in the last 7 days: 0.118343
- Change in the last 90 days: 0.035503 



Question 5: If i had invested into GOTO vs BREN on their respective IPO listing date, which one would have given me a better return over a 90 day horizon?

Invoking: `get_performance_since_ipo` with `{'stock': 'GOTO'}`


https://api.sectors.app/v1/listing-performance/GOTO/
{"symbol": "GOTO.JK", "chg_7d": 0.118343, "chg_30d": -0.343195, "chg_90d": 0.035503, "chg_365d": -0.707101}
Invoking: `get_performance_since_ipo` with `{'stock': 'BREN'}`


https://api.sectors.app/v1/listing-performance/BREN/

In [ ]:
tools = [
    get_company_overview,
    # we created this in the earlier code chunk under Exercise 1
    # (so make sure you've run that cell),
    get_top_companies_by_tx_volume,
    get_daily_tx,
    get_performance_since_ipo
]

query_1 = "What are the top 3 companies by transaction volume over the last 7 days?"
query_2 = "Based on the closing prices of BBCA between 1st and 30th of June 2024, are we seeing an uptrend or downtrend? Try to explain why."
query_3 = "What is the company with the largest market cap between BBCA and BREN? For said company, retrieve the email, phone number, listing date and website for further research."
query_4 = "What is the performance of GOTO (symbol: GOTO) since its IPO listing?"
query_5 = "If i had invested into GOTO vs BREN on their respective IPO listing date, which one would have given me a better return over a 90 day horizon?"

queries = [query_1, query_2, query_3, query_4, query_5]

llm = ChatGroq(
    temperature=0,
    model_name="llama3-groq-70b-8192-tool-use-preview",
    groq_api_key=GROQ_API_KEY,
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"""
            You are the financial robo-advisor, answer the following queries like a Financial Agent,
            being as factual and analytical as possible. Do not make up stuff.
            Try to cite your number as much as possible. Retrieve and show the corresponding data first
            such as the market cap number or data that correspond to the question before you make conclusion and show the answer.
            Your get_daily_tx tool will need the start and end dates, and if you need the start and end dates
            and they are not explicitly provided, please infer from the query.
            If the volume was about a single day (such as specific single date or yesterday),
            the start date and end date parameter should be the same date. Start and end date will be in the format YYYY-MM-DD.
            Whenever you return a list of names, return also the corresponding values for each name.

            For most of your queries such as company performance since the date of its initial public offering (IPO),
            these information should be available in your get_performance_since_ipo tool,
            but the listing performance data is accessible only for stocks listed after May 2005.
            Note that the endpoint for performance since IPO has only one required parameter, which is the stock.
            Every time you return the symbol of the stock, result also the number in question in a nice tabular format.

            Today's date is {get_today_date()}.
            """
        ),
        ("human", "{input}"),
        # msg containing previous agent tool invocations and corresponding tool outputs
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

for i, query in enumerate(queries):
    print(f"Question {i+1}:", query)
    result = agent_executor.invoke({"input": query})
    print("Answer:", "\n", result["output"], "\n\n======\n\n")

Question 1: What are the top 3 companies by transaction volume over the last 7 days?


> Entering new AgentExecutor chain...

Invoking: `get_top_companies_by_tx_volume` with `{'start_date': '2024-07-30', 'end_date': '2024-08-06', 'top_n': 3}`


{"2024-07-30": [{"symbol": "GOTO.JK", "company_name": "PT GoTo Gojek Tokopedia Tbk", "volume": 1223172300, "price": 55}, {"symbol": "BSBK.JK", "company_name": "PT Wulandari Bangun Laksana Tbk", "volume": 792338900, "price": 69}, {"symbol": "BIPI.JK", "company_name": "PT Astrindo Nusantara Infrastruktur Tbk.", "volume": 629061500, "price": 59}], "2024-07-31": [{"symbol": "GOTO.JK", "company_name": "PT GoTo Gojek Tokopedia Tbk", "volume": 2829429300, "price": 53}, {"symbol": "BIPI.JK", "company_name": "PT Astrindo Nusantara Infrastruktur Tbk.", "volume": 555588600, "price": 61}, {"symbol": "BUKA.JK", "company_name": "PT Bukalapak.com Tbk", "volume": 542254600, "price": 118}], "2024-08-01": [{"symbol": "GOTO.JK", "company_name": "PT GoTo Gojek Toko